In [1]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

In [18]:
def get_flight_limit(area_name):
    if "yellow" in area_name.lower():
        return 60
    elif "red" in area_name.lower():
        return 30
    elif "blue" in area_name.lower():
        return 90
    else:
        return -1

In [4]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
restriction_areas = gpd.read_file('flight_restriction_areas.kml', driver='KML')
restriction_areas["boundary"] = restriction_areas.boundary

restriction_areas

,Name,Description,geometry,boundary
0,Yellow area,,"POLYGON Z ((8.92387 44.40400 0.00000, 8.94172 ...","LINESTRING Z (8.92387 44.40400 0.00000, 8.9417..."
1,Red area,,"POLYGON Z ((8.97801 44.40574 0.00000, 8.96061 ...","LINESTRING Z (8.97801 44.40574 0.00000, 8.9606..."
2,Yellow area 2,,"POLYGON Z ((8.95307 44.41890 0.00000, 8.94270 ...","LINESTRING Z (8.95307 44.41890 0.00000, 8.9427..."


In [23]:
limits = []

for area_name in restriction_areas["Name"]:
    limits.append(get_flight_limit(area_name))

restriction_areas["flight_limit"] = limits
restriction_areas

,Name,Description,geometry,boundary,flight_limit
0,Yellow area,,"POLYGON Z ((8.92387 44.40400 0.00000, 8.94172 ...","LINESTRING Z (8.92387 44.40400 0.00000, 8.9417...",60
1,Red area,,"POLYGON Z ((8.97801 44.40574 0.00000, 8.96061 ...","LINESTRING Z (8.97801 44.40574 0.00000, 8.9606...",30
2,Yellow area 2,,"POLYGON Z ((8.95307 44.41890 0.00000, 8.94270 ...","LINESTRING Z (8.95307 44.41890 0.00000, 8.9427...",60


In [5]:
# Get coordinates from boundary
coords = []

for i in range(len(restriction_areas)):
    coords.append(list(restriction_areas.boundary[i].coords))

coords

[[(8.92386666263514, 44.40400146346487, 0.0),
  (8.941718666306686, 44.40032722051951, 0.0),
  (8.946094915742899, 44.41218972364573, 0.0),
  (8.928191157755002, 44.4158055842684, 0.0),
  (8.92386666263514, 44.40400146346487, 0.0)],
 [(8.978009867243307, 44.4057423543738, 0.0),
  (8.960610935268381, 44.4103686912777, 0.0),
  (8.954886885079002, 44.40128213543649, 0.0),
  (8.976984259284306, 44.39610744765416, 0.0),
  (8.978009867243307, 44.4057423543738, 0.0)],
 [(8.953065040412415, 44.41890119822603, 0.0),
  (8.942697603459425, 44.39583632432577, 0.0),
  (8.98180088842816, 44.38818359973404, 0.0),
  (8.988689651618762, 44.40872289477481, 0.0),
  (8.953065040412415, 44.41890119822603, 0.0)]]

In [6]:
# Generate random points within each polygon
points = restriction_areas.sample_points(10)
print(points)

points.explore(m=restriction_areas.explore(), color='red')

0    MULTIPOINT (8.92493 44.40610, 8.92542 44.40754...
1    MULTIPOINT (8.95945 44.40113, 8.96024 44.40403...
2    MULTIPOINT (8.94612 44.39571, 8.95036 44.40644...
Name: sampled_points, dtype: geometry


In [11]:
# Parse the points coordinates to make a string for the request to open elevation API
def parse_points(points):
    coords = []
    coords_string = []

    for i in range(len(points)):
        coords.append(list(points[i].coords))

    for i in range(len(coords)):
        coords[i] = [(x[1], x[0]) for x in coords[i]]
    
    for c in coords:
        coords_string.append(str(c[0][0]) + ',' + str(c[0][1]))
    
    msg = '|'.join(coords_string)

    return msg

payloads = []
for i in range(len(points)):
    payloads.append(parse_points(list(points[i].geoms)))
payloads


['44.40610166279067,8.92493110272219|44.40753605679692,8.925417214262035|44.40953064517299,8.927884030693704|44.41194403774552,8.928683044828734|44.40307542715092,8.930266737492277|44.40324795329047,8.930540904097255|44.40773950096644,8.930585188795767|44.404843925432274,8.940581741457503|44.41008185535484,8.943152599004536|44.407828018826024,8.94378108361349',
 '44.40113340747903,8.959448443860522|44.404029694709976,8.960244192581518|44.40949674673863,8.960397743752324|44.400367433490864,8.963427184873307|44.401081030562274,8.964162610391911|44.40138737745443,8.965280966764473|44.40792784957073,8.967880175745583|44.400213664033636,8.96935777652962|44.40065889300886,8.975437780594994|44.40432257305095,8.977373567848348',
 '44.39570849810507,8.946123434631154|44.40644124553548,8.950363023288245|44.40763099586175,8.950556655979772|44.41075767776386,8.966043701081523|44.40232068407127,8.968166186526272|44.39796687028564,8.973361271743068|44.40725715255062,8.974401912859923|44.400426753404

In [14]:
import requests

def get_elevation(coordinates):
    api_url = 'https://api.open-elevation.com/api/v1/lookup?locations='
    url = api_url + coordinates
    response = requests.get(url)

    elevations = []
    
    if response.status_code == 200:
        data = response.json()
        if 'results' in data and len(data['results']) > 0:
            # Get all elevations and return
            for result in data['results']:
                elevations.append(result['elevation'])
            return elevations
        else:
            print("No results found for the provided coordinates.")
    else:
        print("Error occurred while fetching data:", response.status_code)

# Call the function with the coordinates string
elevations = []
for payload in payloads:
    elevations.append(get_elevation(payload))

elevations


[[0.0, 10.0, 4.0, 12.0, 7.0, 7.0, 31.0, 23.0, 28.0, 22.0],
 [61.0, 58.0, 31.0, 52.0, 52.0, 56.0, 58.0, 53.0, 65.0, 68.0],
 [1.0, 12.0, 12.0, 81.0, 84.0, 56.0, 68.0, 55.0, 43.0, 18.0]]